In [1]:
import pandas as pd
import re, os 
import datetime
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

os.listdir('./data_down')


"""
download data without empty ou, when select
['bca_cc_cust_rel.csv',
 'bca_without_mail.csv',
 'dwd_dsc_d365_contract_df.csv',
 'dwd_dsc_d365_opportunity_df.csv',
 'dwd_dsc_finance_estimate_summary_di.csv',
 'dwd_dsc_hr_cost_dtl.csv',
 'dwd_fact_warehouse_billing_detail_dtl.csv']
"""

"\ndownload data without empty ou, when select\n['bca_cc_cust_rel.csv',\n 'bca_without_mail.csv',\n 'dwd_dsc_d365_contract_df.csv',\n 'dwd_dsc_d365_opportunity_df.csv',\n 'dwd_dsc_finance_estimate_summary_di.csv',\n 'dwd_dsc_hr_cost_dtl.csv',\n 'dwd_fact_warehouse_billing_detail_dtl.csv']\n"

In [23]:
# df_oppa = pd.read_csv('./data_down/' + 'dwd_dsc_d365_opportunity_df.csv', sep = '\001') 
df_bill = pd.read_csv('./data_down/' + 'dwd_dsc_finance_estimate_summary_di.csv', sep = '\001') 
df_cost = pd.read_csv('./data_down/' + 'dwd_dsc_hr_cost_dtl.csv', sep = '\001') 
df_esti = pd.read_csv('./data_down/' + 'dwd_fact_warehouse_billing_detail_dtl.csv', sep = '\001') 
df_bca = pd.read_csv('./data_down/' + 'bca_without_mail.csv', sep = '\001') 
rel = pd.read_csv('./data_down/' + 'bca_cc_cust_rel.csv', sep = '\001') 
# for i in os.listdir('./data_down'): 
#     print(re.findall( '\w+', i)[0])
#     df = pd.read_csv('./data_down/' + i, sep = '\001') 
def colname_modi(df):
    df.columns = [ re.sub('^\w+.{1}', '', i) for i in list(df.columns)]
for i in [df_oppa,df_bill,df_cost,df_esti,rel, df_bca]:
    colname_modi(i)

In [24]:
# pre screen data
df_cost = df_cost.dropna(axis=1, how = 'all').fillna(0)
frame = pd.DataFrame(df_cost.sum(axis = 0) == 0)
df_cost = df_cost.loc[:, frame[frame[0] == False].index.tolist()]
del frame
rel = rel.drop(['cc_in_bca', 'start_date'], axis=1)

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_20484/1032199403.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  frame = pd.DataFrame(df_cost.sum(axis = 0) == 0)


In [5]:
def data_prepare():
    """
    data prepare
    """
    df = pd.read_csv('./data_down/dwd_dsc_d365_opportunity_df.csv', sep = '\001')

    df.columns = [re.sub('^\w+.{1}', '', i) for i in list(df.columns)]
    df['contract_end_date'] = df['contract_end_date'].str.slice(0,10).fillna(pd.NaT)
    colname = ['annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue']
    for i in colname:
        df = df[~df.index.isin(df[i].str.extract('([a-zA-Z]+)').dropna().index)]

    df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue']] = df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue']].astype(float)
    df['contract_end_date'] = pd.to_datetime(df['contract_end_date'])

    contract_summary = df.groupby([
        'account_name_en', 'contract_end_date', 'account_id','bg', 'contract_term', 'milestone','opportunity_record'\
        ,'product1', 'product2', 'sector'
        ])[[
        'annual_average_gross_profit_base', 'annual_average_gross_profit',
        'annual_average_revenue_base', 'annual_average_revenue', 'contract_value']].sum()
    contract_summary = contract_summary.reset_index()
    contract_summary['contract_end_yr'] = contract_summary['contract_end_date'].astype(str).str.slice(0,4)
    return contract_summary

opportunity = data_prepare()


In [6]:
df_list = [df_oppa,df_bill,df_cost,df_esti]

In [7]:
def check_ouname(df):
    print(pd.Series(df.columns).str.extract('(ou_code|cost_center)').dropna(),'\t')

for i in df_list:
    check_ouname(i)

Empty DataFrame
Columns: [0]
Index: [] 	
              0
13  cost_center 	
         0
1  ou_code 	
              0
12  cost_center 	


In [8]:
def to_cust_level(df):
    """
    merge all to cust level
    """
    try:
        df = df.merge(rel, left_on = 'ou_code', right_on = 'cc_in_ou', how  ='left')
    except:
        df = df.merge(rel, left_on = 'cost_center', right_on = 'cc_in_ou', how  ='left')

    return df

In [19]:
to_cust_level(df_oppa)

KeyError: 'cost_center'

In [17]:
bill_cust = to_cust_level(df_bill)


,company_code,company_name,voucher_no,close_date,finance_year,voucher_date,accounting_date,clearing_document_no,amount_of_local_currency,update_currency_general_ledger,...,invoice_date,clear_status,account_name_en,account_name_cn,account_id,dept_ids,cc_in_bca,cc_in_ou,start_date,business_type
0,IL07,顺丰英运科技（上海）有限公司,4400001797,0,2021,20210922,20210922,NaN,12566723.00,CNY,...,20211025.0,部分开票,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IL07,顺丰英运科技（上海）有限公司,4400001789,0,2021,20210922,20210922,NaN,142390.24,CNY,...,20211022.0,全额开票,NaN,NaN,NaN,NaN,NaN,GXLTKGXIHS,2021-06-24,仓储
2,IL07,顺丰英运科技（上海）有限公司,100000667,0,2021,20210930,20210930,NaN,4756.35,CNY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,IL07,顺丰英运科技（上海）有限公司,100000667,0,2021,20210930,20210930,NaN,3957.06,CNY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,IL07,顺丰英运科技（上海）有限公司,100000667,0,2021,20210930,20210930,NaN,8810.29,CNY,...,20211025.0,超额开票,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025,KI13,英运物流（天津）有限公司,4400000072,0,2021,20210927,20210927,NaN,562447.34,CNY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HONEYTJALS,2020-09-02,仓储
2026,KI13,英运物流（天津）有限公司,4400000072,0,2021,20210927,20210927,NaN,70000.00,CNY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HONEYTJALS,2020-09-02,仓储
2027,KI13,英运物流（天津）有限公司,4400000072,0,2021,20210927,20210927,NaN,273082.15,CNY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HONEYTJALS,2020-09-02,仓储
2028,KI13,英运物流（天津）有限公司,4400000072,0,2021,20210927,20210927,NaN,419053.11,CNY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HONEYTJALS,2020-09-02,仓储


In [18]:
df_list
# def get_options(list_stocks):
#     dict_list = []
#     for i in list_stocks:
#         dict_list.append({'label': i, 'value': i})
#     return dict_list

[     opportunity_number                                   opportunity_name  \
 0               Z004029            Renewal of J&J Medical VAS WH -C1102535   
 1               Z004187          Luckin Coffee Logistics Finance 2019-2020   
 2               Z008673  Harley-Davidson COC & MEP Certificates Production   
 3               Z009221                                     Renew_欧克咖啡纸箱销售   
 4               Z009442      Renew_Agfa Hong Kong Limited Contract Renewal   
 ...                 ...                                                ...   
 6619            Z005325    China Mobile 2020-2021 Jiangsu Migu Engineering   
 6620            Z002734            China Mobile Fuzhou warehousing service   
 6621            Z009564                     Renew_Huawei Central Warehouse   
 6622            Z003231                  china mobile hebei transportation   
 6623            Z008482                  内蒙古分公司2020年1月-12月干线物流配送项目服务合同补充协议   
 
                                 account_id    acc

In [ ]:
df_sub = contract_summary

In [ ]:
df_sub[df_sub['account_name_en'] == 'ABC']['annual_average_revenue']

In [ ]:
min = pd.to_datetime('2018-01-01', format='%Y-%m-%d')
max = pd.to_datetime('2024-01-01', format='%Y-%m-%d')

In [ ]:
# fig = go.Figure(data = [go.Scatter(x = df_sub ['contract_end_date'] ,
#                                 y = df_sub ['annual_average_revenue'],
#                                 mode='markers',
#                                 marker_size=df_sub['annual_average_revenue']/1000,
#                                 opacity=0.87,
#                                 name='account',
#                                 textposition='bottom center')],
#                                 layout=go.Layout(
#                                     xaxis = dict(range = [min, max]),
#                                     yaxis = dict(range = [0, 100000]),
#                                     legend = {'xanchor': 'left'}
#                                 )
#                                 ) 
# fig

In [ ]:
# df['contract_end_date'] = df['contract_end_date'].str.slice(0,10).fillna(pd.NaT)
# colname = ['annual_average_gross_profit_base', 'annual_average_gross_profit',
#        'annual_average_revenue_base', 'annual_average_revenue']
# for i in colname:
#     df = df[~df.index.isin(df[i].str.extract('([a-zA-Z]+)').dropna().index)]

# df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
#     'annual_average_revenue_base', 'annual_average_revenue']] = df[['annual_average_gross_profit_base', 'annual_average_gross_profit',
#     'annual_average_revenue_base', 'annual_average_revenue']].astype(float)
# df['contract_end_date'] = pd.to_datetime(df['contract_end_date'])
# contract_summary = df.groupby(['account_name_en', 'contract_end_date'])[[
#        'annual_average_gross_profit_base', 'annual_average_gross_profit',
#        'annual_average_revenue_base', 'annual_average_revenue']].sum()
# contract_summary = contract_summary.reset_index()
# contract_summary['contract_end_yr'] = contract_summary['contract_end_date'].astype(str).str.slice(0,4)

In [ ]:
contract_summary['contract_end_date']

In [ ]:
# fig = px.scatter(x = "contract_end_date", y= "annual_average_revenue", \
#     color = 'account_name_en',  size = 'annual_average_revenue',
#     data_frame  = contract_summary.head(65))

# min = pd.to_datetime('2018-01-01', format='%Y-%m-%d')
# max = pd.to_datetime('2024-01-01', format='%Y-%m-%d')
# fig.update_xaxes(range = list([min, max]))

# fig.update_layout(
#     title_text="Title",
#     showlegend = True
# )

# # Add range slider
# fig.update_layout(

# )

